## CRUD Example w/ Price Alerts

In [1]:
from saxo_apy import SaxoOpenAPIClient
import json

with open("app_config.json", "r") as f:
    config = json.load(f)

client = SaxoOpenAPIClient(config)
client.login()

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID f30f2341df094098b723558db9bf6db2)


In [2]:
# retrieve price alerts
alerts = client.get("/vas/v1/pricealerts/definitions")
print(
    f"These price alerts are currently configured: {[alert for alert in alerts['Data']]}"
)

These price alerts are currently configured: []


In [3]:
# create new price alert: AAPL traded above 200 USD
account_id = client.get("/port/v1/accounts/me")["Data"][0]["AccountId"]

new_alert = {
    "AccountId": account_id,
    "AssetType": "Stock",
    "IsRecurring": False,
    "Operator": "GreaterOrEqual",
    "PriceVariable": "Traded",
    "TargetValue": "200",
    "Uic": 211,
    "ExpiryDate": "2022-12-31T00:00:00Z",
}

created_alert = client.post("/vas/v1/pricealerts/definitions", data=new_alert)
alert_id = created_alert["AlertDefinitionId"]
print(f"New price alert was configured with ID: {alert_id}")

New price alert was configured with ID: 875990


In [4]:
# update price alert (price is now 210 USD)
updated_alert = {
    "AccountId": account_id,
    "AssetType": "Stock",
    "IsRecurring": False,
    "Operator": "GreaterOrEqual",
    "PriceVariable": "Traded",
    "TargetValue": "210",
    "Uic": 211,
    "ExpiryDate": "2022-12-31T00:00:00Z",
}

client.put(f"/vas/v1/pricealerts/definitions/{alert_id}", data=updated_alert)

# no response here because PUT status code is 204

In [5]:
# retrieve updated alert, price is indeed updated to 210
alert = client.get(f"/vas/v1/pricealerts/definitions/{alert_id}")
print(f"Target value of price alert {alert_id} is now: {alert['TargetValue']}")

Target value of price alert 875990 is now: 210.0


In [6]:
# remove alert
client.delete(f"/vas/v1/pricealerts/definitions/{alert_id}")

# again no response because status code is always 204

In [7]:
# alert is indeed removed
client.get("/vas/v1/pricealerts/definitions/")
print(
    f"These price alerts are currently configured: {[alert for alert in alerts['Data']]}"
)

These price alerts are currently configured: []
